In [ ]:
import pandas as pd
import numpy as np

# Melbourne Housing Market

Датасет представляет из себя цены на квартире в мельбурне. Данные взять с kaggle (https://www.kaggle.com/anthonypino/melbourne-housing-market/data).
Я поставил такую цель - научиться предсказывать цену по параметрам недвижимости.
Это может быть актуально при создании объявлений о покупке\аредны жилья. Вряд ли кто-то захочет сильно переплачивать или недополучать за недвижимость, если в 100 метрах рядом есть такое же.
В качестве целевого признака будет использовать Price

In [ ]:
df_raw = pd.read_csv('../data/Melbourne_housing_FULL.csv')
df_raw["Date"] = pd.to_datetime(df_raw["Date"])
df_raw.head()

In [ ]:
df_raw.info()

Как видим, по целевому признаку есть пропуски. Уже есть цель их восстановить:) 
8 - категориальных 
13 - num признаков

Примерный перевод признаков

**Suburb**: Название пригорода

**Address**: Адресс

**Rooms**: Количество комнат

**Price**: Цена в долларах

**Method**: S - property sold; SP - property sold prior; PI - property passed in; PN - sold prior not disclosed; SN - sold not disclosed; NB - no bid; VB - vendor bid; W - withdrawn prior to auction; SA - sold after auction; SS - sold after auction price not disclosed. N/A - price or highest bid not available.

**Type**: br - bedroom(s); h - дом, коттедж, вила; u - блок домов, дуплекс; t - таун хаус; dev site - в процессе постройки; o res - other residential.

**SellerG**: Аген по недвижимости

**Date**: Дата продажи

**Distance**: Расстояние до CBD (Central Business District)

**Regionname**: Регион (West, North West, North, North east ...etc)

**Propertycount**: количество продающихся домов в данном пригороде.

**Bedroom2**: Scraped # of Bedrooms (from different source)

**Bathroom**: Количество ванных комнат

**Car**: Количество машиноместа

**Landsize**: Площадь участка

**BuildingArea**: Размер дома

**YearBuilt**: Год постройки

**CouncilArea**: Какой гос совет тут руководит

**Lattitude, Longtitude** - координаты	

In [ ]:
## Уберем записи с пропущенным price
print(df_raw.shape)

df_missed = df_raw[df_raw['Price'].isnull()]
df = df_raw[df_raw['Price'].notnull()]

print(df_missed.shape, df.shape)

In [ ]:
df_p = df.drop('Price', axis=1)
y = df.Price.values

# Работа с исходными данными и визуацлизаця

In [ ]:
import seaborn as sns
from scipy.stats import normaltest, shapiro, skewtest
import matplotlib.pyplot as plt

## Целевой признак

In [ ]:
sns.distplot(y);

Как видим, у переменной большой хвост справа. Это вполне ожидаемо: любой каприз за ваши деньги. Такую велечину плохо прогнозировать. Попробуем её логорифмировать:

In [ ]:
sns.distplot(np.log(y));

Уже больше похоже на нормальное распределение.

Протестируем на нормальность и скошенность:

In [ ]:
k2, p = normaltest(np.log(y))
print('normaly:', k2, p)

k2, p = skewtest(np.log(y))
print('skewness:', k2, p)

## Тест на нормальность и скошенность не пройден

Построим нормальное с таким же средним и стандартным отклонением

In [ ]:
a = np.random.normal(np.mean(np.log(y)), np.std(np.log(y)), size=len(y))

In [ ]:
sns.distplot(np.log(y));
sns.distplot(a);

Распределения очень похожи. По всей видимости, целевая переменная не проходит тест из-за бимодальности. Резкий всплекс цен приходится на exp(14) ~ 1.2млн$. Возможно это как-то связано с местным законодатесльвом.


**Будем считать, что логорифмированные данные нормальны и не скошенны.**

# Взаимодействие признаков

### Корреляции

In [ ]:
a4_dims = (11.7, 8.27)
fig, ax = plt.subplots(figsize=a4_dims)

sns.heatmap(df.corr(), annot=True, ax=ax);

**Выводы**:

Как видно, сильно скоррелированы Rooms и Bedroom2. Фактически это один и тот же признак, но второй с пропусками.

Так же с количеством комнат скоррелированы - количество ванных комнат, машиномест и цена. Это тоже интуитивно понятно - чем больше жильё, тем больше в нем комнат, ванных, машиномест и тем оно дороже.

Год постройки и дистанция "немного" скоррелированы. Это можно объяснить тем, что в центре уже нет мест для новых домов. И все новое строится далеко.

Чем старше дом - тем он дешевле.

### Попарное распределение

In [ ]:
sns.pairplot(df.dropna());

Если скачать картинку, то лучше будет видно.

Треугольники в правых и верхних углах - это дистанция от центра в зависиомти от координаты.

Рассмотрим второй столбец (цена). Можно заметить а) есть максимумы цены по коориднатам (похоже на центр) 2) Цена не так сильно зависит от размера жилья, как это могло показаться

Есть выбросы по площади постройки, году постройки. Имеет смысл убрать дома до 1200 и площадью более 1500.

### Зависимость цены от типа жилья и времени

In [ ]:
price = df.sort_values("Date", ascending=False).groupby(["Date", 'Type'])[['Price']].mean().reset_index()
price.head()

In [ ]:
sns.pointplot(x='Date', y='Price', data=price, hue='Type');

Явного тренда нет

# Пропуски
Работает с данными, где нет пропусков по цене

In [ ]:
df_p.info()

In [ ]:
df_p[df_p.isnull().any(axis=1)]

Как видим, пропуски идут "пачками". "Bedroom2" из-за скорелированости с Room (было показано ранее). Коориднаты можно брать как средние по району\улице. 

In [ ]:
df_p.drop('Bedroom2', axis=1, inplace=True)
df_p.head()

Ввиду громоздкости это не вошло в ноутбук, так что пусть будет как факт: четкой **связи между** риэлетром\типом постройки и пропусками **нет**. У всех риэлтеров есть пропуски. Их где-то ~50%. Похоже, что это какая-то внутреняя особеность объявлений.

## Восстановление пропусков
Для начала преобразуем формат улицы: вначале идет номер дома, потом название улицы. Уберем номер дома.
Для каждой улицы посчитаем среднюю кооридианту среди непропусов:

In [ ]:
df_p['Address'] = df_p['Address'].apply(lambda x: ' '.join(x.split()[1:]))
df_p.head()

### Пропуски координат
Сделаем их как среднее по улице\району\почтовому коду

In [ ]:
df_full_cor = df_raw[['Suburb', 'Address', 'Postcode', 'Lattitude', 'Longtitude']].dropna()
df_full_cor['Address'] = df_full_cor['Address'].apply(lambda x: ' '.join(x.split()[1:]))
df_full_cor.head()

In [ ]:
def cor_replacer(x, col):
    if not pd.isnull(x[col]):
        return x[col]
    
    if x['Address'] in df_full_cor['Address'].values:
        return df_full_cor[df_full_cor['Address'] == x['Address']][col].mean()
    
    if x['Suburb'] in df_full_cor['Suburb'].values:
        return df_full_cor[df_full_cor['Suburb'] == x['Suburb']][col].mean()
    
    if x['Postcode'] in df_full_cor['Postcode'].values:
        return df_full_cor[df_full_cor['Postcode'] == x['Postcode']][col].mean() 
    
    print(x.Suburb, col)

In [ ]:
df_p['Lattitude'] = df_p.apply(cor_replacer, axis=1, col='Lattitude')
df_p['Longtitude'] = df_p.apply(cor_replacer, axis=1, col='Longtitude')

### Пропуски Bathroom, Car, Landsize, BuildingArea, YearBuilt 
Аналогично прошлому, но с учетом типа жилья

In [ ]:
df_full_cor = df_raw[['Suburb', 'Address', 'Postcode', 'Type', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt']].dropna()
df_full_cor['Address'] = df_full_cor['Address'].apply(lambda x: ' '.join(x.split()[1:]))
df_full_cor.head()

In [ ]:
def value_replacer(x, col):
    if not pd.isnull(x[col]):
        return x[col]
    
    if x['Address'] in df_full_cor['Address'].values:
        return df_full_cor[df_full_cor['Address'] == x['Address']][col].mean()
    
    if x['Suburb'] in df_full_cor['Suburb'].values:
        return df_full_cor[df_full_cor['Suburb'] == x['Suburb']][col].mean()
    
    if x['Postcode'] in df_full_cor['Postcode'].values:
        return df_full_cor[df_full_cor['Postcode'] == x['Postcode']][col].mean() 
    
    print(x.Suburb, col)

In [ ]:
def cor_replacer(x, col):
    if not pd.isnull(x[col]):
        return x[col]
    
    if x['Address'] in df_full_cor['Address'].values:
        return df_full_cor[df_full_cor['Address'] == x['Address']][col].mean()
    
    if x['Suburb'] in df_full_cor['Suburb'].values:
        return df_full_cor[df_full_cor['Suburb'] == x['Suburb']][col].mean()
    
    if x['Postcode'] in df_full_cor['Postcode'].values:
        return df_full_cor[df_full_cor['Postcode'] == x['Postcode']][col].mean() 
    
    print(x.Suburb, col, x.Type)
    return df_full_cor[col].mean()

In [ ]:
df_p['Bathroom'] = df_p.apply(cor_replacer, axis=1, col='Bathroom')
df_p['Car'] = df_p.apply(cor_replacer, axis=1, col='Car')
df_p['Landsize'] = df_p.apply(cor_replacer, axis=1, col='Landsize')
df_p['BuildingArea'] = df_p.apply(cor_replacer, axis=1, col='BuildingArea')
df_p['YearBuilt'] = df_p.apply(cor_replacer, axis=1, col='YearBuilt')

In [ ]:
drop = np.invert(df_p.isnull().any(axis=1))

In [ ]:
df_p[df_p.isnull().any(axis=1)]

# Построение модели
Для тестирования будет использовать 20% данных.
Обучение на 80%.

Т.к. это задача регресии, то в качестве метрики здесь будет использоваться Mean Squared Error. Это связано с тем, что модель не должна допускать больших отклонений от истиной стоимости. В самом деле, предположим, что у нас есть 5 домов, которые стоят по 1000. Пусть первая модель даст предсказания 1000, 500, 1000, 1000, 1000, вторая 900 900 900 900 900. У обеих моделей абсолюная ошибка будет одинаковой (значит, с точки зрения метрик абсолютных ошибок - модели одинаковы), но на деле первая модель хуже. Это связано с тем, что человек, который продал дом в 2 раза дешевле, будет крайне недоволен.

В задаче есть сметь категориальных и количественных признаков. Поэтому для решения будем использовать деревья. Так же деревья не так сильно бояться выбросов и плохого масштаба признаков. 

Данные имеют распределение по времени, так что будет использовать более новые для теста.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, KFold
import xgboost
from sklearn.preprocessing import LabelEncoder 
from collections import defaultdict
from sklearn.metrics import mean_squared_error

## Работа с фичами
1) переботаем дату. Учитывать конкретную неделю\день не имеет смысла: продажа\покупки долгий и заранее планируемый процесс.

In [ ]:
df_p['year'] = df_p['Date'].apply(lambda x: x.year)
df_p['month'] = df_p['Date'].apply(lambda x: x.month)
df_p.drop('Date', axis=1, inplace=True)

In [ ]:
df_p.head()

Уберем 3 фичи с пропусками непонятный природы

In [ ]:
y = np.log(df.Price.values[drop])
X = df_p[drop]

In [ ]:
split_id = int(0.8*X.shape[0])
split_id

Закодируем категориальные данные данные. Кодировщики сохранятся в словаре для дальнейшего использования.

In [ ]:
decoders = defaultdict(LabelEncoder)

In [ ]:
categorial_data = ['Suburb', 'Address', 'Type', 'Method', 'SellerG', 'Postcode', 'CouncilArea', 'Regionname']

In [ ]:
for col in categorial_data:
    print(col)
    X[col] = decoders[col].fit_transform(X[col])

In [ ]:
X[X.isnull().any(axis=1)]

Разделим на тест и треин

In [ ]:
X_train, X_test = X[:split_id], X[split_id:]
y_train, y_test = y[:split_id], y[split_id:]

In [ ]:
X_train.shape, X_test.shape

Поиск лучших гиперпараметров для случайного леса. В случае случайного леса большее количество деревьев не должно ухудать количество предсказаний (проворено, это так). Поэтому будем настраиваеть только глубину, в целях экономии времени. В дальнейшем будет показано, что файнтьюн не особо помогает и нет смысла тратить на машинное него время.


Для кросвалидации будем использовать разделение на 4 фолда с перемешиванием. Это компенсирует эффект заглядывания "в будущее". Cпойлер: дата окажется не очень важной фичей.

In [ ]:
%%time
params = {'n_estimators':np.arange(10, 120, 10),
          'max_depth':np.arange(1, 12, 2)
         }

params = {'max_depth':np.arange(1, 25, 2)
         }

best_model = GridSearchCV(RandomForestRegressor(n_estimators=100, random_state=1984), 
                          params, 
                          scoring='neg_mean_squared_error', 
                          cv=KFold(n_splits=4, random_state=1984, shuffle=True),
                          n_jobs=-1)
best_model.fit(X_test, y_test)

In [ ]:
best_model.best_params_

In [ ]:
%%time

rf_reg = RandomForestRegressor(n_estimators=2000, max_depth=17, n_jobs=-1, random_state=1984)
rf_reg.fit(X_train, y_train)

pred = rf_reg.predict(X_test)

In [ ]:
mean_squared_error(y_test, pred)

В среднем ошибаемся на 15% цены:

In [ ]:
np.mean(np.abs(np.exp(pred) - np.exp(y_test))) / np.exp(y_test).mean()

## Важность фич

In [ ]:
f_impt=pd.DataFrame({'value':rf_reg.feature_importances_,
                     'columns':X_test.columns})
f_impt.head()

In [ ]:
a4_dims = (11.7, 8.27)
fig, ax = plt.subplots(figsize=a4_dims)

g = sns.barplot(x='columns', y='value', data=f_impt, ax=ax);
g.set_xticklabels(g.get_xticklabels(), rotation=90);

Вполне ожидаемо, что наиболее важными фичами оказались Количество комнат, тип жилья и расстояние до центра.

Улица и район имеют малое значение. Возможно, что это компенсированно фичами широты и долготы, а так же индекса, которые тоже отвечают за географическое положение. Вообще, это не одно и тоже, но достаточно близко.

Количество парковочных мест оказалось не слишком важной фичей, что странно.

# Буст
Попробуем более сложную модель дерева. Ради эксперимента, попробуем её переобучить на тестовую выборку.
Поиск оптимальных параметров будет делать с помощью гиперопта.

In [ ]:
from xgboost import XGBClassifier
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import xgboost as xgb

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_test, label=y_test)

In [ ]:
def score(params):
    print("Training with params: ")
    print(params)
    num_round = int(params['n_estimators'])
    del params['n_estimators']
    
    watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
    gbm_model = xgb.train(params, dtrain, num_round,
                          evals=watchlist,
                          verbose_eval=True)
    predictions = gbm_model.predict(dvalid,
                                    ntree_limit=gbm_model.best_iteration + 1)
    #return predictions
    score = mean_squared_error(y_test, predictions)
    # TODO: Add the importance for the selected features
    with open('../export/log_boost.txt', 'a') as f:
        print("\tScore {0}".format(score), num_round, params, file=f, sep='\t')
    print("\tScore {0}\n\n".format(score))
    loss = score
    return {'loss': loss, 'status': STATUS_OK}

In [ ]:
space = {
    'n_estimators': hp.quniform('n_estimators', 30, 350, 1),
    'eta': hp.quniform('eta', 0.025, 0.5, 0.025),
    'max_depth':  hp.choice('max_depth', np.arange(1, 14, dtype=int)),
    'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
    'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
    'gamma': hp.quniform('gamma', 0.5, 1, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'eval_metric': 'rmse',
    'objective': 'reg:linear',
    'nthread': 7,
    'booster': 'gbtree',
    'silent': 1,
    'seed': 1984
}

In [ ]:
best = fmin(score, space, algo=tpe.suggest, max_evals=100)

In [ ]:
space_best = space
for k, v in best.items():
    space_best[k] = v
    
num_round = int(space_best['n_estimators'])
del space_best['n_estimators']

space_best

In [ ]:
gbm_model = xgb.train(space_best, dtrain, num_round,
                      verbose_eval=False)
gbm_predictions = gbm_model.predict(dvalid, ntree_limit=gbm_model.best_iteration + 1)

In [ ]:
np.mean((np.abs(np.exp(gbm_predictions) - np.exp(y_test)))/np.exp(y_test))

Примерно, тот же результат, что и у леса

# Вывод
Как можно заметить, использование переобученного буста не дало значимого прироста результата.
Таким образом, можно использовать обычный случайный лес (увелчение количества деревьев не дает значимого прироста результатов). 

Думаю, что часть ошибки связана с параметрами, которые не входят в исходный датасет: срочность продажи, состояния здания, ремонт, субъективность оценки, ожидание каких-то новых событий (например, метро через пару лет).

Модель можно применять на сайтах продажи недвижимости: после заполнения параметров недвижимости, продавцу будет показываться ориентировачная цена. 

Другой вариант: сервис по защите от мошенников. Приложение\сайт будет показывать ориентировочную цену жилья в этом райное, что защитит покупателей\продавцов от нечестный предложений.